In [ ]:
!pip install firebase

In [11]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from firebase_admin import credentials, initialize_app, db
# Set up Firebase connection
cred = credentials.Certificate('/content/drive/MyDrive/Colab Notebooks/CloudCourse/Tirgul6/cloudtirgul6-firebase-adminsdk-3m2sb-5d58089424.json')
initialize_app(cred, {'databaseURL': 'https://cloudtirgul6-default-rtdb.firebaseio.com/'})

In [35]:
# Function to fetch questions from Firebase
def fetch_questions():
    global questions_list
    questions_data = questions_ref.get()
    questions_list = [{'key': key, **value} for key, value in questions_data.items()]


In [36]:
# Function to display main menu
def display_main_menu():
    clear_output(wait=True)
    
    add_question_button = widgets.Button(description="Add a Question")
    play_game_button = widgets.Button(description="Play the Game")

    display(add_question_button)
    display(play_game_button)

    def on_add_question_click(button):
        admin_input()

    def on_play_game_click(button):
        clear_output(wait=True)
        fetch_questions()
        display_game()

    add_question_button.on_click(on_add_question_click)
    play_game_button.on_click(on_play_game_click)

In [42]:
# Function to prompt admin for adding a question
def admin_input():
    clear_output(wait=True)
    
    print("Enter the question and answers:")
    question_input = widgets.Text(description="Question")
    correct_answer_input = widgets.Text(description="Correct Answer", style={'description_width':'115px'})
    incorrect_answers_input = [widgets.Text(description=f"Incorrect Answer {i + 1}", style={'description_width':'115px'}) for i in range(3)]
    submit_button = widgets.Button(description="Submit")
    cancel_button = widgets.Button(description="Cancel")

    display(question_input)
    display(correct_answer_input)
    for answer_input in incorrect_answers_input:
        display(answer_input)
    display(submit_button)
    display(cancel_button)

    def on_submit_click(button):
        question = question_input.value
        correct_answer = correct_answer_input.value
        answers = [answer_input.value for answer_input in incorrect_answers_input]
        add_question(question, correct_answer, *answers)
        display_main_menu()

    def on_cancel_click(button):
        display_main_menu()

    submit_button.on_click(on_submit_click)
    cancel_button.on_click(on_cancel_click)

# Function to add a question
def add_question(question, correct_answer, *answers):
    question_data = {
        'question': question,
        'correct_answer': correct_answer,
        'answers': [correct_answer, *answers]
    }
    questions_ref.push(question_data)

In [44]:
# Function to display game
def display_game():
    if questions_list:
        question_data = questions_list.pop(random.randint(0, len(questions_list) - 1))
        display_question(question_data)
    else:
        print(f"Game over! You scored {points} points.")
        print(f"Correct answers: {correct_answers}")
        print(f"Incorrect answers: {incorrect_answers}")
        print("Click the button below to restart the game.")
        restart_button = widgets.Button(description="Restart")
        display(restart_button)

        def on_restart_click(button):
            global questions_list, points, correct_answers, incorrect_answers
            clear_output(wait=True)
            points = 0
            correct_answers = 0
            incorrect_answers = 0
            fetch_questions()
            display_game()

        restart_button.on_click(on_restart_click)

# Function to display a question
def display_question(question_data):
    global points, correct_answers, incorrect_answers
    question = question_data['question']
    correct_answer = question_data['correct_answer']
    answers = question_data['answers']
    random.shuffle(answers)

    print(question)
    buttons = [widgets.Button(description=answer, layout=widgets.Layout(width='auto')) for answer in answers]
    end_game_button = widgets.Button(description="End Game", button_style='danger')

    def on_button_click(button):
        global points, correct_answers, incorrect_answers
        if button.description == correct_answer:
            print("\033[92mCorrect!\033[0m")
            points += 10
            correct_answers += 1
        else:
            print("\033[91mIncorrect!\033[0m")
            incorrect_answers += 1
        time.sleep(2.5)
        clear_output(wait=True)
        display_game()

    def on_end_game_click(button):
        clear_output(wait=True)
        display_end_game()

    for button in buttons:
        display(button)
        button.on_click(on_button_click)
    display(end_game_button)
    end_game_button.on_click(on_end_game_click)

# Function to display end game results and pie chart
def display_end_game():
    print(f"Game over! You scored {points} points.")
    print(f"Correct answers: {correct_answers}")
    print(f"Incorrect answers: {incorrect_answers}")

    if correct_answers + incorrect_answers > 0:
        labels = 'Correct', 'Incorrect'
        sizes = [correct_answers, incorrect_answers]
        colors = ['green', 'red']
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        plt.axis('equal')
        plt.show()
    else:
        print("No questions were answered.")
    
    print("Click the button below to restart the game.")
    restart_button = widgets.Button(description="Restart")
    display(restart_button)

    def on_restart_click(button):
        global questions_list, points, correct_answers, incorrect_answers
        clear_output(wait=True)
        points = 0
        correct_answers = 0
        incorrect_answers = 0
        fetch_questions()
        display_main_menu()

    restart_button.on_click(on_restart_click)

In [45]:
# Import necessary libraries
from IPython.display import display, clear_output
import ipywidgets as widgets
import random
import time
import matplotlib.pyplot as plt

# Define global variables
questions_ref = db.reference('questions')
questions_list = []
points = 0
correct_answers = 0
incorrect_answers = 0

# Start the main menu
display_main_menu()

Button(description='Add a Question', style=ButtonStyle())

Button(description='Play the Game', style=ButtonStyle())